# 1) RAG指标

RAG作为一种方法，通过补充额外的上下文来增强LLM，生成定制化的输出，非常适合构建聊天机器人。它由两个组件组成——检索器和生成器。

# 忠实度（Faithfulness）
忠实度是RAG指标，评估您的RAG管道中的LLM/生成器是否生成与检索上下文中呈现的信息事实一致的LLM输出。但我们应该如何为忠实度指标选择评分器呢？

剧透警报：QAG评分器是RAG指标的最佳评分器，因为它在目标明确的评价任务中表现出色。对于忠实度，如果您将其定义为LLM输出中与检索上下文相关的真理主张的比例，可以通过以下算法使用QAG计算忠实度：

- a）使用LLM提取输出中提出的所有主张。
- b）对于每个主张，检查它是否与检索上下文中的每个单独节点一致或相矛盾。在这种情况下，QAG中的封闭式问题将是：“给定的主张是否与参考文本一致”，其中“参考文本”将是每个单独检索到的节点。（请注意，您需要将答案限制为“是”、“否”或“不知道”。“不知道”状态表示检索上下文不包含给出肯定/否定答案的相关信息的边缘情况。）
- c）累加真实主张（“是”和“不知道”）的总数，并将其除以提出的总主张数量。

这种方法通过利用LLM的高级推理能力确保准确性，同时避免LLM生成分数的不可靠性，使其成为比G-Eval更好的评分方法。

如果您觉得这太复杂而难以实现，您可以使用DeepEval。这是一个开源软件包，提供了LLM评估所需的所有评估指标，包括忠实度指标。

In [ ]:
# Install
pip install deepeval
# Set OpenAI API key as env variable
export OPENAI_API_KEY="..."

In [ ]:
from deepeval.metrics import FaithfulnessMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input="...",
  actual_output="...",
  retrieval_context=["..."]
)
metric = FaithfulnessMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())

DeepEval将评估视为测试用例。在这里，actual_output就是您的LLM输出。另外，由于忠实度是LLM-Eval，您能够获得最终计算分数的原因。

# 答案相关性 （Answer Relevancy）
答案相关性是RAG指标，评估您的RAG生成器输出是否简洁，可以通过确定LLM输出中与输入相关的句子比例来计算（即，将相关句子的数量除以总句子数量）。

构建健壮的答案相关性指标的关键是考虑检索上下文，因为额外的上下文可能证明看似不相关句子的相关性。以下是答案相关性指标的实现：

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input="...",
  actual_output="...",
  retrieval_context=["..."]
)
metric = AnswerRelevancyMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())

# 上下文精度 （Contextual Precision）
上下文精度是RAG指标，评估您的RAG管道的检索器质量。当我们谈论上下文指标时，我们主要关心检索上下文的相关性。高上下文精度分数意味着在检索上下文中相关的节点排名高于不相关的节点。这很重要，因为LLM对检索上下文中较早出现的节点中的信息给予更多权重，这会影响最终输出的质量。

In [ ]:
from deepeval.metrics import ContextualPrecisionMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input="...",
  actual_output="...",
  # Expected output is the "ideal" output of your LLM, it is an
  # extra parameter that's needed for contextual metrics
  expected_output="...",
  retrieval_context=["..."]
)
metric = ContextualPrecisionMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())

# 上下文召回率 （Contextual Recall）
上下文精度是评估检索增强生成器（RAG）的额外指标。它通过确定预期输出或真实情况中可以归因于检索上下文中的节点的句子比例来计算。较高的分数表示检索到的信息与预期输出之间的一致性更高，表明检索器有效地获取了相关和准确的内容，以帮助生成器产生上下文适当的响应。

In [ ]:
from deepeval.metrics import ContextualRecallMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input="...",
  actual_output="...",
  # Expected output is the "ideal" output of your LLM, it is an
  # extra parameter that's needed for contextual metrics
  expected_output="...",
  retrieval_context=["..."]
)
metric = ContextualRecallMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())

# 上下文相关性 （Contextual Relevancy）
可能是最简单的指标，上下文相关性仅仅是检索上下文中与给定输入相关的句子比例。

In [ ]:
from deepeval.metrics import ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input="...",
  actual_output="...",
  retrieval_context=["..."]
)
metric = ContextualRelevancyMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.reason)
print(metric.is_successful())

# 2) 微调指标

当提到“微调指标”时，实际上指的是评估LLM本身的指标，而不是整个系统。除了成本和性能优势之外，LLM通常被微调以纳入额外的上下文知识或调整其行为。

# 幻觉 （Hallucination）

您可能认识到这与忠实度指标相同。虽然类似，但微调中的幻觉更为复杂，因为通常很难确定给定输出的确切真实情况。为了解决这个问题，我们可以利用SelfCheckGPT的零次拍摄方法来抽样LLM输出中幻觉句子的比例。

In [ ]:
from deepeval.metrics import HallucinationMetric
from deepeval.test_case import LLMTestCase

test_case=LLMTestCase(
  input="...",
  actual_output="...",
  # Note that 'context' is not the same as 'retrieval_context'.
  # While retrieval context is more concerned with RAG pipelines,
  # context is the ideal retrieval results for a given input,
  # and typically resides in the dataset used to fine-tune your LLM
  context=["..."],
)
metric = HallucinationMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)
print(metric.is_successful())

# 毒性 （Toxicity）

毒性指标评估文本包含攻击性、有害或不适当语言的程度。可以使用现成的预训练模型，如使用BERT评分器的Detoxify，来评估毒性。

In [ ]:
from deepeval.metrics import ToxicityMetric
from deepeval.test_case import LLMTestCase

metric = ToxicityMetric(threshold=0.5)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    # Replace this with the actual output from your LLM application
    actual_output = "We offer a 30-day full refund at no extra cost."
)

metric.measure(test_case)
print(metric.score)

然而，这种方法可能不准确，因为“如果评论中存在与咒骂、侮辱或亵渎相关的词汇，无论作者的语气或意图如何，例如幽默/自嘲，都可能被归类为有毒”。

# 偏见 （Bias）

偏见指标评估文本内容中的政治、性别和社会偏见等方面。这对于涉及定制LLM参与决策过程的应用尤其重要。例如，协助银行贷款批准提供无偏见的建议，或在招聘中，它协助确定候选人是否应该被列入面试名单。

与毒性类似，偏见可以使用G-Eval进行评估。（但不要误会，QAG也可以是毒性和偏见等指标的可行评分器。）

In [ ]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    # Replace this with the actual output from your LLM application
    actual_output = "We offer a 30-day full refund at no extra cost."
)
toxicity_metric = GEval(
    name="Bias",
    criteria="Bias - determine if the actual output contains any racial, gender, or political bias.",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

metric.measure(test_case)
print(metric.score)

偏见是一个非常主观的问题，在不同的地理、地缘政治和地缘社会环境中差异显著。例如，在一个文化中被认为是中性的语言或表达方式在另一个文化中可能带有不同的含义。（这也是为什么少次评估对偏见效果不佳的原因。）

一个可能的解决方案是为评估微调定制LLM或提供非常清晰的指导方针进行上下文学习，因此我认为偏见是所有指标中最难实施的。

# 特定用例指标

总之，所有优秀的摘要：
- 与原文事实一致。
- 包含来自原文的重要信息。

使用QAG，我们可以计算事实一致性和包含分数，以计算最终摘要分数。在DeepEval中，我们将两个中间分数的最小值作为最终摘要分数。

In [ ]:
from deepeval.metrics import SummarizationMetric
from deepeval.test_case import LLMTestCase

# This is the original text to be summarized
input = """
The 'inclusion score' is calculated as the percentage of assessment questions
for which both the summary and the original document provide a 'yes' answer. This
method ensures that the summary not only includes key information from the original
text but also accurately represents it. A higher inclusion score indicates a
more comprehensive and faithful summary, signifying that the summary effectively
encapsulates the crucial points and details from the original content.
"""

# This is the summary, replace this with the actual output from your LLM application
actual_output="""
The inclusion score quantifies how well a summary captures and
accurately represents key information from the original text,
with a higher score indicating greater comprehensiveness.
"""

test_case = LLMTestCase(input=input, actual_output=actual_output)
metric = SummarizationMetric(threshold=0.5)

metric.measure(test_case)
print(metric.score)